In [1]:
import pandas as pd
import json

In [3]:
business_data = json.load(open("yelp_dataset/yelp_filtered_philadelphia_business.json"))

# Create a DataFrame for business data
business_df = pd.DataFrame(business_data)

# Flatten the nested "attributes" column
business_attributes = pd.json_normalize(business_df['attributes'])
business_attributes.columns = [f'attributes.{col}' for col in business_attributes.columns]

business_hours = pd.json_normalize(business_df['hours'])
business_hours.columns = [f'WorkingHours.{col}' for col in business_hours.columns]


# Concatenate the flattened attributes DataFrame with the original business DataFrame
business_df = pd.concat([business_df, business_attributes], axis=1)
business_df = pd.concat([business_df, business_hours], axis=1)

# Drop the original nested "attributes" column
business_df = business_df.drop('attributes', axis=1)
business_df = business_df.drop('hours', axis=1)

In [6]:
review_data = json.load(open("yelp_dataset/yelp_filtered_philadelphia_reviews.json"))
review_df = pd.DataFrame(review_data)

### Each review is unique

In [8]:
review_df["review_id"].duplicated().sum()

0

In [10]:
business_df["is_open"].value_counts()

1    10542
0     4027
Name: is_open, dtype: int64

### Filter permanently closed businesses

In [11]:
business_df = business_df[business_df["is_open"] == 1]

In [12]:
business_df.drop(columns=["city", "state", "latitude", "longitude", "is_open"], inplace=True)
review_df.drop(columns=["review_id", "user_id", "date"], inplace=True)

### Filter reviews for businesses that are still open

In [13]:
review_df = review_df[review_df["business_id"].isin(business_df["business_id"])]